# Phase 4v2: CoT 기반 인기글 생성

## 📌 목적
Phase 3v2에서 저장한 파인튜닝 모델을 로드하여,
RPM 팩터 정보와 CoT(Thinking → Response) 구조를 활용한 인기글을 생성합니다.

### 기존 대비 개선점
- ✅ 팩터(인기유형) 정보를 입력으로 제공 → 의도한 스타일 생성
- ✅ CoT: 모델이 먼저 전략을 사고(Thinking) → 그에 맞게 글 생성(Response)
- ✅ EOS 토큰 학습 완료 → 자연스러운 생성 종료

## ⚙️ 사전 조건
- `phase2_dataset_v2.ipynb` 실행으로 `outputs/rpm_factors.json` 존재
- `phase3_finetune_v2.ipynb` 실행으로 `models/llama3_popular_post_lora_v2` 존재
- GPU 런타임 할당

---
## 0. 패키지 설치

In [9]:
# ── Phase 4 패키지 설치 (최종) ────────────────────────────
import os
os.environ["WANDB_DISABLED"] = "true"

# unsloth + unsloth_zoo 최신 안정 버전 강제 재설치
%pip install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo
%pip install --upgrade pyarrow

print("✅ 패키지 설치 완료 — 런타임을 재시작하세요!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 414.3 MB/s eta 0:00:00
  Attempting uninstall: unsloth_zoo
    Found existing installation: unsloth_zoo 2026.2.1
    Uninstalling unsloth_zoo-2026.2.1:
      Successfully uninstalled unsloth_zoo-2026.2.1
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2026.2.1
    Uninstalling unsloth-2026.2.1:
      Successfully uninstalled unsloth-2026.2.1


✅ 패키지 설치 완료 — 런타임을 재시작하세요!


In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")

Mounted at /content/drive
✅ Google Drive 마운트 완료


---
## 1. 경로 설정 및 팩터 정보 로드

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"
import unsloth  # 반드시 맨 위에서 import

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

In [3]:
from pathlib import Path
import json
import pandas as pd
from IPython.display import display

PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR   = PROJECT_ROOT / "outputs"
MODEL_DIR    = PROJECT_ROOT / "models"

# ── RPM 팩터 정보 로드 ────────────────────────────────────
FACTORS_PATH = OUTPUT_DIR / "rpm_factors.json"
if FACTORS_PATH.exists():
    with open(FACTORS_PATH, "r", encoding="utf-8") as f:
        factors_data = json.load(f)
    factor_names = [f["factor_name"] for f in factors_data["factors"]]
    factor_stats = factors_data.get("factor_stats", {})
    print(f"✅ 팩터 정보 로드 완료: {len(factor_names)}개 팩터")
    for name in factor_names:
        stats = factor_stats.get(name, {})
        coverage = stats.get("coverage", 0)
        print(f"  → {name} (커버리지: {coverage:.0%})")
else:
    print("⚠️ 팩터 정보 없음, 기본 모드로 생성합니다.")
    factor_names = ["일반"]
    factor_stats = {}

# ── 토픽 키워드 로드 ──────────────────────────────────────
PROMPT_FILES = {
    "익게2":     OUTPUT_DIR / "익게2_topics_for_prompt.json",
    "자유게시판": OUTPUT_DIR / "자유게시판_topics_for_prompt.json",
    "연애상담소": OUTPUT_DIR / "연애상담소_topics_for_prompt.json",
    "익게1":     OUTPUT_DIR / "익게1_topics_for_prompt.json",
}

trend_keywords = {}
for board, path in PROMPT_FILES.items():
    if not path.exists():
        print(f"⚠️ {board}: {path} 없음 → 건너뜀")
        continue
    with open(path, "r", encoding="utf-8") as f:
        topics_data = json.load(f)
    top_topics = sorted(topics_data, key=lambda x: len(x.get("representatives", [])), reverse=True)[:5]
    keywords = []
    for topic in top_topics:
        kw = topic.get("keywords", [])
        if isinstance(kw, list):
            keywords.extend(kw[:8])
    trend_keywords[board] = list(dict.fromkeys(keywords))[:8]
    print(f"✅ {board}: {len(trend_keywords[board])}개 키워드")

if not trend_keywords:
    raise RuntimeError("키워드가 없습니다. Phase 1 노트북을 먼저 실행하세요.")

✅ 팩터 정보 로드 완료: 6개 팩터
  → 학내 정체성 및 소속감 강화 (커버리지: 32%)
  → 진로 및 실용적 정보의 전문성 (커버리지: 7%)
  → 사회적 현안에 대한 비판적 담론 (커버리지: 12%)
  → 정서적 공감과 인간적 유대 (커버리지: 17%)
  → 유희적 재미와 서사적 카타르시스 (커버리지: 8%)
  → 커뮤니티 특화 소통 및 서사 전략 (커버리지: 11%)
✅ 익게2: 8개 키워드
✅ 자유게시판: 8개 키워드
⚠️ 연애상담소: /content/drive/MyDrive/board_crawling/outputs/연애상담소_topics_for_prompt.json 없음 → 건너뜀
⚠️ 익게1: /content/drive/MyDrive/board_crawling/outputs/익게1_topics_for_prompt.json 없음 → 건너뜀


---
## 2. 파인튜닝 모델 로드

In [4]:
from unsloth import FastLanguageModel
import torch

FINETUNED_DIR = MODEL_DIR / "llama3_popular_post_lora_v2"
if not FINETUNED_DIR.exists():
    raise FileNotFoundError(f"{FINETUNED_DIR}가 없습니다. Phase 3v2를 먼저 실행하세요.")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=str(FINETUNED_DIR),
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)
print("✅ 파인튜닝 모델 로드 완료")

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ 파인튜닝 모델 로드 완료


---
## 3. CoT 기반 생성 함수

In [5]:
def generate_post_v2(
    board_name: str,
    keywords: list,
    factor_name: str = "일반",
    temperature: float = 0.7,
    max_new_tokens: int = 1024,
) -> dict:
    """
    CoT 기반 인기글 생성.
    모델이 Thinking → Response를 순차 생성하고, 각각 분리하여 반환합니다.
    """
    # 팩터 통계 정보
    stats = factor_stats.get(factor_name, {})
    coverage = stats.get("coverage", 0)
    factor_info = f"{factor_name} (커버리지: {coverage:.0%})" if factor_name != "일반" else "일반"

    instruction = f"주어진 키워드와 인기요인 정보를 사용하여 {board_name}의 인기글 스타일로 글을 작성해줘."
    input_text = f"키워드: {', '.join(keywords)}\n인기요인: {factor_info}"

    BOS = tokenizer.bos_token
    prompt = (
        f"{BOS}Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{instruction}\n\n"
        f"### Input:\n{input_text}\n\n"
        f"### Thinking:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Llama-3 EOS 토큰 설정
    eos_ids = [tokenizer.eos_token_id]
    eot_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")
    if eot_id != tokenizer.unk_token_id:
        eos_ids.append(eot_id)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=eos_ids,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # ── Thinking / Response 분리 ──────────────────────────
    thinking = ""
    response = ""

    if "### Thinking:" in generated_text and "### Response:" in generated_text:
        parts = generated_text.split("### Response:")
        response = parts[-1].strip()

        thinking_part = parts[0]
        if "### Thinking:" in thinking_part:
            thinking = thinking_part.split("### Thinking:")[-1].strip()
    elif "### Response:" in generated_text:
        response = generated_text.split("### Response:")[-1].strip()
    else:
        response = generated_text.strip()

    # ── 제목/본문 분리 ────────────────────────────────────
    lines = response.strip().split("\n")
    if len(lines) > 1:
        title = lines[0].strip()
        content = "\n".join(lines[1:]).strip()
    else:
        title = response[:50].strip() + "..." if len(response) > 50 else response.strip()
        content = response.strip()

    return {
        "board_name": board_name,
        "keywords": ", ".join(keywords),
        "factor": factor_name,
        "thinking": thinking,
        "generated_title": title,
        "generated_content": content,
    }


print("✅ CoT 기반 생성 함수 정의 완료")

✅ CoT 기반 생성 함수 정의 완료


---
## 4. 게시판별 + 팩터별 생성 실행

In [6]:
from datetime import datetime

generated_posts = []

# 각 게시판별로, 상위 2개 팩터에 대해 생성
top_factors = factor_names[:3] if len(factor_names) >= 3 else factor_names

for board, keywords in trend_keywords.items():
    if not keywords:
        continue

    for factor in top_factors:
        print(f"\n🔧 생성 중: [{board}] 팩터={factor}")
        print(f"   키워드: {', '.join(keywords[:5])}...")

        try:
            result = generate_post_v2(
                board_name=board,
                keywords=keywords,
                factor_name=factor,
                temperature=0.7,
                max_new_tokens=1024,
            )
            result["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            generated_posts.append(result)

            print(f"   ✅ 생성 완료")
            print(f"   💭 Thinking: {result['thinking'][:100]}...")
            print(f"   📝 제목: {result['generated_title']}")
            print(f"   📄 내용: {result['generated_content'][:150]}...")

        except Exception as exc:
            print(f"   ⚠️ 생성 실패: {exc}")


🔧 생성 중: [익게2] 팩터=학내 정체성 및 소속감 강화
   키워드: 계엄, 대통령, 계엄령, 속보, 이...
   ✅ 생성 완료
   💭 Thinking: 국가적 비상사태인 계엄 상황에서 헌법과 법률에 대한 근본적인 질문을 던짐으로써 학생들의 정치적 관심사를 자극했습니다. 공적 가치에 민감한 대학생 집단 내에서 사회 현안에 대해 날카...
   📝 제목: 계엄!?
   📄 내용: 왜 국회의원들 안 죽이는거야?ㅋㅋㅋㅋㅋ반대 표決して 안 먹힐 거 같은데 왜 계엄한다고 바로 반대표 박는 것도 아니잖아국회도 아닌데 국군은 어케 불러오냐??저런 놈들은 다 싹다 처형해야지…그냥 평소처럼 일하던대로 계속 하다가 저녁때쯤에 한강이나 그럴 때까지 기다려라...

🔧 생성 중: [익게2] 팩터=진로 및 실용적 정보의 전문성
   키워드: 계엄, 대통령, 계엄령, 속보, 이...
   ✅ 생성 완료
   💭 Thinking: 국가적인 비상사태인 계엄 상황이 중대한 정치적 문제라는 점에 주목하며 대중들의 정서적 공감과 비판 의식을 동시에 자극했습니다. 시의성 있는 사회 현안을 다룸으로써 커뮤니티 내에서 ...
   📝 제목: [속보] 윤 대통령, 국회 계엄 반대 지시
   📄 내용: 어차피 저런 사람이 나라 만들 거 같은데계엄령 선포하고 바로 국회 막아라 이거 맞음?지금 당장 군대 태울 시간도 안되는데......

🔧 생성 중: [익게2] 팩터=사회적 현안에 대한 비판적 담론
   키워드: 계엄, 대통령, 계엄령, 속보, 이...
   ✅ 생성 완료
   💭 Thinking: 국가적인 위기 상황에서 대통령의 행동을 강력하게 저격하며 대중의 공분과 분노를 자극했습니다. 사태의 본질을 직시하는 날카로운 비평이 이용자들 사이의 활발한 논쟁으로 이어지며 인기를...
   📝 제목: 계엄령 속보) 한 대통령…?
   📄 내용: ㅋㅋㅋㅋㅋㅅㅂㅋㅋㅋ왜 안 죽음? ㅠㅠㅠㅠ...

🔧 생성 중: [자유게시판] 팩터=학내 정체성 및 소속감 강화
   키워드: 구합니다, 혹시, 수업, 분, 있

---
## 5. 생성 결과 저장 및 비교

In [7]:
if generated_posts:
    result_df = pd.DataFrame(generated_posts)
    output_csv = OUTPUT_DIR / "generated_posts_v2.csv"
    result_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"\n✅ 생성 결과 저장 완료: {output_csv}")

    print("\n" + "=" * 70)
    print("생성 결과 요약".center(70))
    print("=" * 70)
    display(result_df[["board_name", "factor", "generated_title"]].head(20))
else:
    print("⚠️ 생성된 게시글이 없습니다.")


✅ 생성 결과 저장 완료: /content/drive/MyDrive/board_crawling/outputs/generated_posts_v2.csv

                               생성 결과 요약                               


,board_name,factor,generated_title
0,익게2,학내 정체성 및 소속감 강화,계엄!?
1,익게2,진로 및 실용적 정보의 전문성,"[속보] 윤 대통령, 국회 계엄 반대 지시"
2,익게2,사회적 현안에 대한 비판적 담론,계엄령 속보) 한 대통령…?
3,자유게시판,학내 정체성 및 소속감 강화,혹시 더 나은 수업 만들고 싶은 분?
4,자유게시판,진로 및 실용적 정보의 전문성,[국장님] 취준생들을 위한 현직자분 강연회 신청 안내
5,자유게시판,사회적 현안에 대한 비판적 담론,혹시 이런 분 계신가요?


In [8]:
# ── 상세 출력: Thinking + Response ────────────────────────
print("\n" + "=" * 70)
print("CoT 상세 출력 (Thinking → Response)".center(70))
print("=" * 70)

for i, post in enumerate(generated_posts[:5], 1):
    print(f"\n[{i}] [{post['board_name']}] 팩터: {post['factor']}")
    print(f"    키워드: {post['keywords']}")
    print(f"\n    💭 Thinking:")
    print(f"    {post['thinking'][:200]}")
    print(f"\n    📝 Response:")
    print(f"    제목: {post['generated_title']}")
    print(f"    내용: {post['generated_content'][:300]}")
    print("-" * 70)


                   CoT 상세 출력 (Thinking → Response)                    

[1] [익게2] 팩터: 학내 정체성 및 소속감 강화
    키워드: 계엄, 대통령, 계엄령, 속보, 이, 반대, 나라, 아님

    💭 Thinking:
    국가적 비상사태인 계엄 상황에서 헌법과 법률에 대한 근본적인 질문을 던짐으로써 학생들의 정치적 관심사를 자극했습니다. 공적 가치에 민감한 대학생 집단 내에서 사회 현안에 대해 날카로운 분석과 토론이 활발히 이루어지며 높은 주목도를 기록했습니다.

    📝 Response:
    제목: 계엄!?
    내용: 왜 국회의원들 안 죽이는거야?ㅋㅋㅋㅋㅋ반대 표決して 안 먹힐 거 같은데 왜 계엄한다고 바로 반대표 박는 것도 아니잖아국회도 아닌데 국군은 어케 불러오냐??저런 놈들은 다 싹다 처형해야지…그냥 평소처럼 일하던대로 계속 하다가 저녁때쯤에 한강이나 그럴 때까지 기다려라
----------------------------------------------------------------------

[2] [익게2] 팩터: 진로 및 실용적 정보의 전문성
    키워드: 계엄, 대통령, 계엄령, 속보, 이, 반대, 나라, 아님

    💭 Thinking:
    국가적인 비상사태인 계엄 상황이 중대한 정치적 문제라는 점에 주목하며 대중들의 정서적 공감과 비판 의식을 동시에 자극했습니다. 시의성 있는 사회 현안을 다룸으로써 커뮤니티 내에서 활발한 토론과 담론 형성이 가능하게 했습니다.

    📝 Response:
    제목: [속보] 윤 대통령, 국회 계엄 반대 지시
    내용: 어차피 저런 사람이 나라 만들 거 같은데계엄령 선포하고 바로 국회 막아라 이거 맞음?지금 당장 군대 태울 시간도 안되는데...
----------------------------------------------------------------------

[3] [익게2] 팩터: 사회적 현안에

---
## 6. 추가 팁
- 생성 결과를 수작업으로 검토하여 품질 확인 후 배포하세요.
- `temperature`, `factor_name`, 키워드 조합을 조정하여 다양한 샘플을 생성할 수 있습니다.
- 특정 팩터를 지정하면 해당 인기유형에 맞는 글이 생성됩니다.
- CoT의 Thinking 부분이 생성 전략을 보여주므로, 모델의 추론 과정을 해석할 수 있습니다.